### Data Preprocessing

First, let us load the datasets containing metrics per method.

In [ ]:
import os
import pandas as pd

dataset_path = "../../dataset"

directory = os.fsencode(dataset_path)
file_list = os.listdir(directory)
dfs = []

for file in file_list:
    filename = os.fsdecode(file)
    dfs.append(pd.read_csv(f"{dataset_path}\{filename}"))

Let's make sense of the data we're dealing with by inspecting one of the datasets containing collected metrics from a repository by SourceMeter.

In [3]:
from ydata_profiling import ProfileReport

report = ProfileReport(dfs[0])
os.makedirs(r"outputs\data_exploration", exist_ok=True)
# report.to_file(r"outputs\data_exploration\report.html")

First, let us check if there are any NaN values:

In [3]:
for i in range(len(dfs)):
    nan_number = dfs[i].isna().sum().sum()
    if nan_number > 0:
        print(f"We have {nan_number} nan values in df {i}.")

We have 1 nan values in df 13.
We have 1 nan values in df 21.


1 nan value is nothing serious, and it can be dropped.

In [4]:
dfs[13].dropna(inplace=True)
dfs[21].dropna(inplace=True)

Now, let's get rid of all the columns containing no information about metrics, and rescale the existing metric data with MinMaxScaler in order to prepare it for the ABC feature selection process.

In [5]:
from sklearn.preprocessing import MinMaxScaler

def df_transformation(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(["Name", "LongName", "Parent", "Component", "Path", "Line", "Column", "EndLine", "EndColumn", "ID"],
            axis=1, inplace=True)
    scaler = MinMaxScaler()
    scaler.fit_transform(df)
    return df

for df in dfs:
    df = df_transformation(df)

### ABC 🐝🐝🐝 algorithm application

We will apply the ABC algorithm to reduce each dataset to an optimal subset of features of all possible cardinalities, such that the Sammon error is decreased as much as possible. 

In [6]:
import numpy as np
from tqdm import tqdm

def sammon_error(high_dim_data, low_dim_data):
    high_distances = np.linalg.norm(high_dim_data[:, None] - high_dim_data, axis=2)
    low_distances = np.linalg.norm(low_dim_data[:, None] - low_dim_data, axis=2)
    high_dist_sum = np.sum(high_distances)
    sammon_error_value = np.sum(((high_distances - low_distances) ** 2) / (high_distances + 1e-9)) / high_dist_sum
    return sammon_error_value

def optimized_sammon_error(high_distances, low_dim_data):
    low_distances = np.linalg.norm(low_dim_data[:, None] - low_dim_data, axis=2)
    high_dist_sum = np.sum(high_distances)
    sammon_error_value = np.sum(((high_distances - low_distances) ** 2) / (high_distances + 1e-9)) / high_dist_sum
    return sammon_error_value

def reduce_features(data, binary_vector):
    selected_features = data[:, binary_vector == 1]
    return selected_features

# Function performing scramble mutation in order to preserve the cardinality
# of the reduced feature subset
# If we roll less than the mutation rate, then we permute the minimal defined range of permutations
# Since we test the fitness function of a new neighbour solution every iteration
# there is no point in not performing the mutation at all
def mutation(solution, MR, rng, minimum_range):
    generated_range = rng.choice(len(solution), size=2, replace=False)
    generated_range.sort()
    mutation_range = (generated_range[1] - generated_range[0]) / len(solution)
    if mutation_range > MR:
        min_choice = rng.choice(int(np.floor((1-minimum_range)*len(solution))), size=1)[0]
        generated_range = [min_choice, int(min_choice+np.floor(minimum_range*len(solution)))]
        # print(generated_range)
    permute = []
    for i in range(generated_range[0], generated_range[1]):
        permute.append(solution[i])
    np.random.shuffle(permute)
    for i in range(generated_range[0], generated_range[1]):
        solution[i] = permute.pop(0)
    return solution

def abc_run(num_food_sources, MR, MAX_LIMIT, max_iterations, feature_count, num_features, minimum_range, dataset: pd.DataFrame):
    rng = np.random.default_rng()
    food_sources = []
    for i in range(num_food_sources):
        pos_indices = rng.choice(num_features, size=feature_count, replace=False)
        food_sources.append([1 if i in pos_indices else 0 for i in range(num_features)])
    food_sources = np.array(food_sources)
    limits = np.zeros(num_food_sources)
    best_solution = None
    best_error = float('inf')
    error_history = []
    dataset_norms = np.linalg.norm(dataset[:, None] - dataset, axis=2)
    low_dim_errors = np.zeros(num_food_sources)

    for iteration in tqdm(range(max_iterations)):
        for i in range(num_food_sources):
            subset_data = reduce_features(dataset, food_sources[i])
            if low_dim_errors[i] == 0:
                current_error = optimized_sammon_error(dataset_norms, subset_data)
                low_dim_errors[i] = current_error
            else:
                current_error = low_dim_errors[i]
            neighbor = food_sources[i].copy()
            neighbor = mutation(neighbor, MR, rng, minimum_range)

            neighbor_subset_data = reduce_features(dataset, neighbor)
            neighbor_error = optimized_sammon_error(dataset_norms, neighbor_subset_data)

            if neighbor_error < current_error:
                food_sources[i] = neighbor
                low_dim_errors[i] = neighbor_error
                limits[i] = 0
            else:
                limits[i] += 1

            if current_error < best_error:
                best_solution = food_sources[i]
                best_error = current_error

        for i in range(num_food_sources):
            if limits[i] >= MAX_LIMIT:
                pos_indices = rng.choice(num_features, size=feature_count, replace=False)
                food_sources[i]=np.array([1 if i in pos_indices else 0 for i in range(num_features)])
                limits[i] = 0

        error_history.append(best_error)

    return best_solution, error_history

In [7]:
param_combinations = {
    'num_sources': [5, 10, 20],
    'mr': [0.3, 0.4, 0.5],
    'max_limit': [3, 5, 10],
    'iterations': [20, 50, 100],
    'minimum_range': [0.05, 0.1, 0.15]
}

Trying to find optimal parameters for every setting for every dataset would be overly expensive (we would have to launch it 27*27 times), so we'll launch it once for the dfs[0] with a reduction to 13 features - middle of the road reduction. We'll use these hyperparameters for all the other settings as well, getting our first results. 

In [18]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

class ABCOptimizer(BaseEstimator, RegressorMixin):
    def __init__(self, num_sources=10, mr=0.5, max_limit=5, iterations=100, feature_count = 13,
                 minimum_range=0.1, num_features=21):
        self.num_sources = num_sources
        self.mr = mr
        self.max_limit = max_limit
        self.iterations = iterations
        self.feature_count = feature_count
        self.minimum_range = minimum_range
        self.num_features = num_features

    def fit(self, X, y=None):
        self.error_history_ = abc_run(self.num_sources, self.mr, self.max_limit, self.iterations,
                                     self.feature_count, self.num_features, self.minimum_range, X)
        self.final_error_ = self.error_history_[-1]
        return self.final_error_
    
    def score(self, X, y=None):
        return -self.final_error_


In [17]:
optimal_parameters = {}
for feature_count in tqdm(range(2, len(dfs[0].columns))):
    abc_model = ABCOptimizer(feature_count=feature_count, num_features=len(dfs[0].columns))
    grid_search = GridSearchCV(abc_model, param_combinations, cv=3, n_jobs=-1, verbose=3)
    grid_search.fit(dfs[0].values)
    optimal_parameters[feature_count] = f"Optimal_parameters: {grid_search.best_params_}\nBest error: {-grid_search.best_score_}"

  0%|          | 0/25 [4:08:42<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
abc_run(20, 0.5, 5, 50, 2, )

In [ ]:
abc_model = ABCOptimizer(feature_count=2, num_features=len(dfs[0].columns))
grid_search = GridSearchCV(abc_model, param_combinations, cv=2, n_jobs=6, verbose=3)
grid_search.fit(dfs[0].values)
optimal_parameters[feature_count] = f"Optimal_parameters: {grid_search.best_params_}\nBest error: {-grid_search.best_score_}"

Fitting 2 folds for each of 243 candidates, totalling 486 fits


In [ ]:
results = {}
for num_sources in param_combinations['num_food_sources']:
    for mr in param_combinations['MR']:
        for max_limit in param_combinations['MAX_LIMIT']:
            for iterations in param_combinations['max_iterations']:
                for range in param_combinations['minimum_range']
                key = f"num_food_sources={num_sources}, MR={mr}, MAX_LIMIT={max_limit}, iterations={iterations}"
                print(f"Running with: {key}")
                error_history = abc_run(num_sources, mr, max_limit, iterations)
                results[key] = error_history